In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from urllib.parse import urlparse, parse_qs
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from tensorflow.keras.utils import to_categorical
from sklearn.pipeline import make_pipeline
from sklearn.utils import resample
from scipy.sparse import hstack
from sklearn.preprocessing import StandardScaler
from collections import Counter
import math
import re
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.feature_selection import VarianceThreshold
from sklearn.naive_bayes import GaussianNB  
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler  
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.feature_selection import VarianceThreshold
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler 
import socket



# Load the dataset
df = pd.read_csv('C:\\Users\\User\\Desktop\\url_dataset_updated.csv')

# Drop all duplicates from df
df = df.drop_duplicates()

# Reset the index after dropping duplicates
df = df.reset_index(drop=True)

url_column_name = 'URL'  

# Function to concatenate "https://" to benign URLs 
def add_https(url, label):
    if label == 0:
        return "https://" + url
    else:
        return url

# Apply the function to the URL column
df[url_column_name] = df.apply(lambda row: add_https(row[url_column_name], row['Label']), axis=1)


# Separate the dataset into malicious and benign
malicious_df = df[df['Label'] == 1]
benign_df = df[df['Label'] == 0]

# Randomly sample 150,000 entries from each
malicious_sampled_df = resample(malicious_df, n_samples=150000, random_state=42)
benign_sampled_df = resample(benign_df, n_samples=150000, random_state=42)

# Combine the sampled data
balanced_df = pd.concat([malicious_sampled_df, benign_sampled_df])

# Shuffle the combined dataset to mix malicious and benign URLs
balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)



# Now, balanced_df contains the balanced dataset ready for further processing

In [2]:
def ensure_scheme(url):
    if not urlparse(url).scheme:
        url = 'http://' + url
    return url

# Feature extraction functions
def get_url_length(url):
    return len(url)

def get_dot_count(url):
    return url.count('.')

def contains_security_sensitive_words(url):
    security_sensitive_words = [
    'login', 'password', 'admin', 'root', 'secret', 'private', 'secure', 'confidential', 
    'bank', 'creditcard', 'account', 'authentication', 'authorization', 'session', 'token', 
    'apikey', 'ssl', 'https', 'secure', 'encrypted', 'auth', 'signin', 'signup', 'verification', 
    'resetpassword', 'change-password', 'forgot-password', 'otp', '2fa', 'phishing', 'malware', 
    'virus', 'trojan', 'exploit', 'hacker', 'attack', 'security', 'vulnerable', 'injection', 
    'xss', 'csrf', 'dos', 'ddos', 'bruteforce', 'firewall', 'vpn', 'proxy', 'tor', 'security-question', 
    'privacy-policy'
]

    return int(any(word in url for word in security_sensitive_words))

def get_directory_length(url):
    url = ensure_scheme(url)
    path = urlparse(url).path
    return len(path)

def get_sub_directory_count(url):
    url = ensure_scheme(url)
    path = urlparse(url).path
    return path.count('/') - 1

def get_token_count_in_path(url):
    url = ensure_scheme(url)
    path = urlparse(url).path
    tokens = path.split('/')
    return len(tokens) - 1

def get_largest_token_length(url):
    url = ensure_scheme(url)
    path = urlparse(url).path
    tokens = path.split('/')
    if tokens:
        return max(len(token) for token in tokens)
    return 0

def get_average_token_length(url):
    url = ensure_scheme(url)
    path = urlparse(url).path
    tokens = [token for token in path.split('/') if token]
    if tokens:
        return np.mean([len(token) for token in tokens])
    return 0

def get_file_length(url):
    url = ensure_scheme(url)
    path = urlparse(url).path
    filename = path.split('/')[-1]
    return len(filename)

def get_dot_count_in_file(url):
    url = ensure_scheme(url)
    path = urlparse(url).path
    filename = path.split('/')[-1]
    return filename.count('.')

def get_delimiter_count_in_file(url):
    path = urlparse(url).path
    filename = path.split('/')[-1]
    delimiters = ['.', '_', '-']
    return sum(filename.count(delimiter) for delimiter in delimiters)

def get_arguments_length(url):
    url = ensure_scheme(url)
    query = urlparse(url).query
    return len(query)

def get_number_of_arguments(url):
    url = ensure_scheme(url)
    query = urlparse(url).query
    return len(parse_qs(query))

def get_length_of_largest_argument_value(url):
    url = ensure_scheme(url)
    query = urlparse(url).query
    params = parse_qs(query)
    if params:
        return max(len(max(values, key=len)) for values in params.values())
    return 0

def get_max_delimiters_in_arguments(url):
    url = ensure_scheme(url)
    query = urlparse(url).query
    params = parse_qs(query)
    delimiters = ['&', '=', '-', '_']
    if params:
        return max(sum(value.count(delimiter) for delimiter in delimiters) for values in params.values() for value in values)
    return 0


def get_hyphen_count_in_domain(url):
    url = ensure_scheme(url)
    domain = urlparse(url).netloc
    return domain.count('-')

def contains_ip(url):
    url = ensure_scheme(url)
    domain = urlparse(url).netloc
    try:
        socket.inet_aton(domain)
        return 1
    except:
        return 0
    
def get_domain_features(url):
    url = ensure_scheme(url)
    domain = urlparse(ensure_scheme(url)).netloc
    tokens = domain.split('.')
    
    # Domain Length
    domain_length = len(domain)
    
    # Count of Tokens in the Domain
    token_count = len(tokens)
    
    # Length of Largest Token in the Domain
    largest_token_length = max(len(token) for token in tokens) if tokens else 0
    
    # Average Domain Token Length
    average_token_length = sum(len(token) for token in tokens) / len(tokens) if tokens else 0
    
    return domain_length, token_count, largest_token_length, average_token_length

# New feature extraction functions
def get_special_character_count(url):
    special_characters = ['@', '=', '+', '*', '?', '&', '%', '$', '#', '!']
    return sum(url.count(char) for char in special_characters)

def get_entropy(url):
    # Count the frequency of each character in the string
    freq = Counter(url)
    # Calculate the probabilities
    probs = [count / len(url) for count in freq.values()]
    # Calculate the Shannon entropy
    entropy = -sum(p * math.log(p, 2) for p in probs if p > 0)
    return entropy

def check_url_shortened(url):
    shortened_services = ['bit.ly', 'tinyurl.com', 'goo.gl', 'ow.ly', 't.co']
    url = ensure_scheme(url)
    domain = urlparse(url).netloc
    return int(domain in shortened_services)

def get_subdomain_count(url):
    url = ensure_scheme(url)
    domain_parts = urlparse(url).netloc.split('.')
    # Count as subdomains any parts beyond the second-level domain and TLD
    return max(0, len(domain_parts) - 2)

def get_suspicious_tld(url):
    suspicious_tlds = ['xyz', 'top', 'loan', 'win', 'club']
    url = ensure_scheme(url)
    tld = urlparse(url).netloc.split('.')[-1]
    return int(tld in suspicious_tlds)

def get_numeric_ratio(url):
    numeric_chars = sum(c.isdigit() for c in url)
    return numeric_chars / len(url) if len(url) > 0 else 0

def get_word_count(url):
    words = re.findall(r'\w+', url)
    return len(words)

def is_https(url, timeout=0.5):
    return int(url.startswith("https"))


# Apply feature extraction
features = balanced_df['URL'].apply(lambda x: pd.Series({
    'url_length': get_url_length(x),
    'dot_count': get_dot_count(x),
    'hyphen_count_domain': get_hyphen_count_in_domain(x),
    'security_sensitive_words': contains_security_sensitive_words(x),
    'directory_length': get_directory_length(x),
    'sub_directory_count': get_sub_directory_count(x),
    'token_count_path': get_token_count_in_path(x),
    'largest_token_length': get_largest_token_length(x),
    'average_token_length': get_average_token_length(x),
    'file_length': get_file_length(x),
    'contains_ip': contains_ip(x),
    'dot_count_in_file': get_dot_count_in_file(x),
    'delimiter_count_in_file': get_delimiter_count_in_file(x),
    'arguments_length': get_arguments_length(x),
    'number_of_arguments': get_number_of_arguments(x),
    'length_of_largest_argument_value': get_length_of_largest_argument_value(x),
    'max_delimiters_in_arguments': get_max_delimiters_in_arguments(x),
    'special_character_count': get_special_character_count(x),
    'entropy': get_entropy(x),
    'url_shortened': check_url_shortened(x),
    'subdomain_count': get_subdomain_count(x),
    'suspicious_tld': get_suspicious_tld(x),
    'numeric_ratio': get_numeric_ratio(x),
    'domain_length': get_domain_features(x)[0],
    'domain_token_count': get_domain_features(x)[1],
    'largest_domain_token_length': get_domain_features(x)[2],
    'average_domain_token_length': get_domain_features(x)[3],
    'word_count': get_word_count(x),
    'is_https': is_https(x)
}))


# Concatenate original DF with features
balanced_df = pd.concat([balanced_df, features], axis=1)


# Extracting TF-IDF features from URLs
tfidf_vectorizer = TfidfVectorizer(max_features=5000)  # Limiting to top 5000 features
tfidf_features = tfidf_vectorizer.fit_transform(balanced_df['URL'])

# Convert TF-IDF features from a sparse matrix to a dense format and then to an np.ndarray
tfidf_dense = np.asarray(tfidf_features.todense())

# Define X for numerical features
X_numerical = balanced_df.drop(['Label', 'URL'], axis=1).values  # Make sure this matches your feature extraction output

# Combining TF-IDF features with numerical features
X_combined = np.hstack((X_numerical, tfidf_dense))

# Define y
y = balanced_df['Label'].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)





In [17]:
balanced_df

,URL,Label,url_length,dot_count,hyphen_count_domain,security_sensitive_words,directory_length,sub_directory_count,token_count_path,largest_token_length,...,url_shortened,subdomain_count,suspicious_tld,numeric_ratio,domain_length,domain_token_count,largest_domain_token_length,average_domain_token_length,word_count,is_https
0,http://etransfers.interac.ca-ssl.net/sh/2o05I9...,1,65.0,4.0,1.0,1.0,29.0,3.0,4.0,12.0,...,0.0,2.0,0.0,0.061538,29.0,4.0,10.0,6.500000,11.0,0.0
1,http://betterhealthsmoothies.com/Adobe/adobe-3...,1,58.0,2.0,0.0,0.0,26.0,2.0,3.0,9.0,...,0.0,0.0,0.0,0.034483,25.0,2.0,21.0,12.000000,8.0,0.0
2,http://lloydsbank.deregister-payee-secure-auth...,1,60.0,3.0,3.0,1.0,10.0,0.0,1.0,9.0,...,0.0,1.0,0.0,0.000000,43.0,3.0,28.0,13.666667,9.0,0.0
3,https://archive.md,0,18.0,1.0,0.0,1.0,0.0,-1.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,10.0,2.0,7.0,4.500000,3.0,1.0
4,https://pkg00-atx.netgate.com,0,29.0,2.0,1.0,1.0,0.0,-1.0,0.0,0.0,...,0.0,1.0,0.0,0.068966,21.0,3.0,9.0,6.333333,5.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,https://infomation-fb-service.e82443.repl.co,1,44.0,3.0,2.0,1.0,0.0,-1.0,0.0,0.0,...,0.0,2.0,0.0,0.113636,36.0,4.0,21.0,8.250000,7.0,1.0
299996,https://img-1000736.ad-score.com,0,32.0,2.0,2.0,1.0,0.0,-1.0,0.0,0.0,...,0.0,1.0,0.0,0.218750,24.0,3.0,11.0,7.333333,6.0,1.0
299997,http://sosyalsat.com/help/home.html,1,35.0,2.0,0.0,0.0,15.0,1.0,2.0,9.0,...,0.0,0.0,0.0,0.000000,13.0,2.0,9.0,6.000000,6.0,0.0
299998,https://storageapi.fleek.co/12678f8a-04f9-4b69...,1,83.0,3.0,0.0,1.0,56.0,1.0,2.0,43.0,...,0.0,1.0,0.0,0.289157,19.0,3.0,10.0,5.666667,12.0,1.0


In [18]:
# Model Training with Logistic Regression
model = LogisticRegression(random_state=42, max_iter=10000)  # Increased max_iter for convergence in case of large dataset
model.fit(X_train, y_train)

# Predictions and Evaluation
y_pred = model.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.97525
Confusion Matrix:
 [[29828   234]
 [ 1251 28687]]
Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.99      0.98     30062
           1       0.99      0.96      0.97     29938

    accuracy                           0.98     60000
   macro avg       0.98      0.98      0.98     60000
weighted avg       0.98      0.98      0.98     60000



In [19]:
# Model Training with GaussianNB
model = GaussianNB()
model.fit(X_train, y_train)

# Predictions and Evaluation
y_pred = model.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.95715
Confusion Matrix:
 [[28957  1105]
 [ 1466 28472]]
Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.96      0.96     30062
           1       0.96      0.95      0.96     29938

    accuracy                           0.96     60000
   macro avg       0.96      0.96      0.96     60000
weighted avg       0.96      0.96      0.96     60000



In [3]:
# Model Training with RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predictions and Evaluation
y_pred = model.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.9786
Confusion Matrix:
 [[29782   280]
 [ 1004 28934]]
Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.99      0.98     30062
           1       0.99      0.97      0.98     29938

    accuracy                           0.98     60000
   macro avg       0.98      0.98      0.98     60000
weighted avg       0.98      0.98      0.98     60000



In [4]:
model = DecisionTreeClassifier(random_state=42)
model.fit(X_train, y_train)

# Predictions and Evaluation
y_pred = model.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.9743166666666667
Confusion Matrix:
 [[29458   604]
 [  937 29001]]
Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.98      0.97     30062
           1       0.98      0.97      0.97     29938

    accuracy                           0.97     60000
   macro avg       0.97      0.97      0.97     60000
weighted avg       0.97      0.97      0.97     60000



In [5]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_combined)  


model = KNeighborsClassifier(n_neighbors=490)  
model.fit(X_train, y_train)

# Predictions and Evaluation
y_pred = model.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.9156333333333333
Confusion Matrix:
 [[29438   624]
 [ 4438 25500]]
Classification Report:
               precision    recall  f1-score   support

           0       0.87      0.98      0.92     30062
           1       0.98      0.85      0.91     29938

    accuracy                           0.92     60000
   macro avg       0.92      0.92      0.92     60000
weighted avg       0.92      0.92      0.92     60000



In [15]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import to_categorical

# Tokenization and sequence padding parameters
max_len = 200  
max_words = 60000  

# Tokenize the URLs
tokenizer = Tokenizer(num_words=max_words, char_level=True)
tokenizer.fit_on_texts(balanced_df['URL'])
sequences = tokenizer.texts_to_sequences(balanced_df['URL'])

# Pad the sequences
data = pad_sequences(sequences, maxlen=max_len)

# Labels
labels = np.asarray(balanced_df['Label'])
labels = to_categorical(labels)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)


# Model definition
model = Sequential()
model.add(Embedding(max_words, 32, input_length=max_len))
model.add(Bidirectional(LSTM(64, return_sequences=True)))  
model.add(Dropout(0.5)) 
model.add(Bidirectional(LSTM(32))) 
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.001)))  
model.add(Dropout(0.5))  
model.add(Dense(2, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

# Early stopping
early_stopping = EarlyStopping(monitor='val_acc', patience=3, restore_best_weights=True)

# Train the model with early stopping
history = model.fit(X_train, y_train, epochs=30, batch_size=128, validation_split=0.2, callbacks=[early_stopping])

# Predictions
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

# Evaluation
print("Accuracy:", accuracy_score(y_test_classes, y_pred_classes))
print("Confusion Matrix:\n", confusion_matrix(y_test_classes, y_pred_classes))
print("Classification Report:\n", classification_report(y_test_classes, y_pred_classes))

Epoch 1/30
1500/1500 [==============================] - 333s 220ms/step - loss: 0.2025 - acc: 0.9287 - val_loss: 0.1453 - val_acc: 0.9508
Epoch 2/30
1500/1500 [==============================] - 320s 213ms/step - loss: 0.1346 - acc: 0.9550 - val_loss: 0.1157 - val_acc: 0.9608
Epoch 3/30
1500/1500 [==============================] - 319s 213ms/step - loss: 0.1160 - acc: 0.9614 - val_loss: 0.1034 - val_acc: 0.9634
Epoch 4/30
1500/1500 [==============================] - 319s 213ms/step - loss: 0.1116 - acc: 0.9627 - val_loss: 0.0953 - val_acc: 0.9661
Epoch 5/30
1500/1500 [==============================] - 319s 213ms/step - loss: 0.0972 - acc: 0.9674 - val_loss: 0.0953 - val_acc: 0.9659
Epoch 6/30
1500/1500 [==============================] - 319s 213ms/step - loss: 0.0905 - acc: 0.9696 - val_loss: 0.0883 - val_acc: 0.9693
Epoch 7/30
1500/1500 [==============================] - 319s 213ms/step - loss: 0.0852 - acc: 0.9718 - val_loss: 0.0825 - val_acc: 0.9716
Epoch 8/30
1500/1500 [============